Ufuk Altun

In [5]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.3" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

yInstalling Julia 1.6.3 on the current Colab Runtime...
2021-11-07 09:52:17 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.3-linux-x86_64.tar.gz [113399344/113399344] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing Julia package CUDA...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Success! Please reload this page and jump to the next section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Install Knet

In [3]:
using Pkg; 
pkg"add Knet IterTools MLDatasets"

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
   Installed CEnum ────────────── v0.4.1
   Installed ImageMagick_jll ──── v6.9.10-12+3
   Installed LLVMExtra_jll ────── v0.0.11+0
   Installed ExprTools ────────── v0.1.6
   Installed BinaryProvider ───── v0.5.10
   Installed ImageCore ────────── v0.9.3
   Installed Blosc_jll ────────── v1.21.0+0
   Installed URIParser ────────── v0.4.1
   Installed OpenSpecFun_jll ──── v0.5.5+0
   Installed PyCall ───────────── v1.92.5
   Installed GZip ─────────────── v0.5.1
   Installed FileIO ───────────── v1.11.2
   Installed AbstractFFTs ─────── v1.0.1
   Installed LLVM ─────────────── v4.6.0
   Installed Knet ─────────────── v1.4.9
   Installed JLD2 ─────────────── v0.4.15
   Installed Lz4_jll ──────────── v1.9.3+0
   Installed CodecZlib ────────── v0.7.0
   Installed CUDA ─────────────── v3.5.0
   Installed MAT ──────────────── v0.10.1
   Installed ImageMagick ──────── v1.2.2
   Installed HDF5 ────────────

In [ ]:
!gdown --id 1skwblH1_okBwxWxmRsp9_qi15hyPpxg8 #install dataset from avaliable drive link
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')
!tar -xvf  '/content/VLCS.tar.gz' -C '/content/DGvGS/data/orig'

In [3]:
# Install packages before first run: using Pkg; pkg"add Knet IterTools MLDatasets"
using Knet, IterTools, MLDatasets

# Define convolutional layer:
struct Conv; w; b; end
Conv(w1,w2,nx,ny) = Conv(param(w1,w2,nx,ny), param0(1,1,ny,1))
(c::Conv)(x) = relu.(pool(conv4(c.w, x) .+ c.b))

# Define dense layer:
struct Dense; w; b; f; end
Dense(i,o; f=identity) = Dense(param(o,i), param0(o), f)
(d::Dense)(x) = d.f.(d.w * mat(x) .+ d.b)

# Define a chain of layers and a loss function:
struct Chain; layers; end
(c::Chain)(x) = (for l in c.layers; x = l(x); end; x)
(c::Chain)(x,y) = nll(c(x),y)

# Load MNIST data:
xtrn,ytrn = MNIST.traindata(Float32); ytrn[ytrn.==0] .= 10
xtst,ytst = MNIST.testdata(Float32);  ytst[ytst.==0] .= 10
dtrn = minibatch(xtrn, ytrn, 100; xsize = (28,28,1,:))
dtst = minibatch(xtst, ytst, 100; xsize = (28,28,1,:))

# Define and train LeNet (~10 secs on a GPU or ~3 mins on a CPU to reach ~99% accuracy)
LeNet = Chain((Conv(5,5,1,20), Conv(5,5,20,50), Dense(800,500,f=relu), Dense(500,10)))
progress!(adam(LeNet, ncycle(dtrn,3)))
accuracy(LeNet,data=dtst)

SyntaxError: ignored